## Limpieza de datos para el dataset

In [ ]:
!pip install pycountry
!pip install thefuzz[speedup]

In [ ]:
import pandas as pd
import numpy as np
import pycountry
from thefuzz import fuzz
import pycountry

In [ ]:
data_dir = r"data\Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv"
data = pd.read_csv(data_dir, sep="\t")
data.head()

## Estandarizar Nombres de campos

cambio de los nombres de la base de datos a unos mas maleables, ya que los "originales" son muy largos

In [ ]:
new_col_names = ["Timestamp",
"Age",
"Industry",
"Job_title",
"Job_additional_context",
"annual_salary",
"monetary_compensation",
"currency",
"other_currency",
"income_aditional_context",
"job_country",
"us_state",
"job_city",
"professional_experience",
"field_experience",
"highest_education_level",
"gender",
"race"
]


dict_columns = dict(zip(data.columns, new_col_names))
data.columns = [dict_columns[x] for x in data.columns]

In [ ]:
data.head()

## Estandarizacion currency

nueva columna para realizar el proceso de estandarizacion del valor "currency" esta estandarizacion se realiza a través de un diccionario que transforma valores "originales" a valores estandarizados

In [ ]:
c0 = data["currency"].str.upper() == "OTHER"
data["standard_currency"] = data["currency"].copy()
data.loc[c0,"standard_currency"] = data.loc[c0,"other_currency"]

data["currency"] = data["currency"].str.upper()
data["standard_currency"] = data["standard_currency"].str.upper()
data["standard_currency"] = data["standard_currency"].fillna("N/A")


In [ ]:
currency_values = ['AMERICAN DOLLARS', 
        'ARGENTINE PESO', 'ARGENTINIAN PESO (ARS)', 'ARS',
       'AUD', 'AUD AUSTRALIAN ', 'AUD/NZD', 'AUSTRALIAN DOLLARS ', 
       'BDT',
       'BR$', 'BRL', 'BRL (R$)', 
       'CAD', 
       'CHF', 
       'CHINA RMB', 'CNY', 
       'COP',
       'CROATIAN KUNA', 
       'CZECH CROWNS', 'CZK', 
       'DANISH KRONER', 'DKK', 'DKK ',
       'EQUITY', 
       'EUR', 'EURO', 
       'GBP', 
       'HKD', 
       'IDR', 'IDR ', 
       'ILS', 'ILS (SHEKEL)', 'ILS/NIS',       
       'INDIAN RUPEES', 'INR', 'INR (INDIAN RUPEE)',
       'ISRAELI SHEKELS', 
       'JPY', 
       'KOREAN WON ', 'KRW', 'KRW (KOREAN WON)',
       'LKR', 
       'MEXICAN PESOS', 'MXN', 
       'MYR', 
       'NGN', 
       'NIS (NEW ISRAELI SHEKEL)',
       'NOK', 'NORWEGIAN KRONER (NOK)', 
       'NTD', 
       'NZD', 
       'PESO ARGENTINO',
       'PHILIPPINE PESO', 'PHILIPPINE PESO (PHP)', 'PHILIPPINE PESOS', 'PHP','PHP (PHILIPPINE PESO)', 
       'PLN', 'PLN (POLISH ZLOTY)', 'PLN (ZWOTY)', 'POLISH ZŁOTY', 
       'RM', 'RMB (CHINESE YUAN)', 
       'RUPEES', 
       'SAR', 
       'SEK',
       'SGD', 'SGD ', 'SINGAPORE DOLLARA', 
       'TAIWANESE DOLLARS', 'THAI  BAHT', 'THAI BAHT ', 'THB', 
       'TRY', 
       'TTD', 
       'US DOLLAR', 'USD', 
       'ZAR',
       'N/A']

standard_currency_values = ["USD",
                            "ARS","ARS","ARS",
                            "AUD","AUD","AUD","AUD",
                            "BDT",
                            "BRL","BRL","BRL",
                            "CAD",
                            "CHF",
                            "CNY","CNY",
                            "COP",
                            "HRK",
                            "CZK","CZK",
                            "DKK", "DKK", "DKK",
                            "N/A",
                            "EUR","EUR",
                            "GBP",
                            "HKD",
                            "IDR", "IDR",
                            "ILS","ILS","ILS",
                            "INR","INR","INR",
                            "ILS",
                            "JPY",
                            "KRW","KRW","KRW",
                            "LKR",
                            "MXN","MXN",
                            "MYR",
                            "NGN",
                            "ILS",
                            "NOK","NOK",
                            "NTD",
                            "NZD",
                            "ARS",
                            "PHP","PHP","PHP","PHP","PHP",
                            "PLN", "PLN", "PLN", "PLN",
                            "CNY","CNY",
                            "INR",
                            "SAR", 
                            "SEK",
                            "SGD", "SGD", "SGD",
                            "THB", "THB", "THB", "THB",
                            "TRY", 
                            "TTD",
                            "USD", "USD",
                            "ZAR",
                            "N/A"]


dict_currencies = dict(zip(currency_values,standard_currency_values))

#data["standard_currency"] = 

c0 = ~data["currency"].isna()
data.loc[c0,"standard_currency"] = data.loc[c0,"standard_currency"].apply(lambda x:dict_currencies[x] )


## Estandarizacion Paises

se realizan varias iteraciones para estandarizar el nombre de los paises. a lo largo de cada iteracion se ira construyendo un diccionario que tendra las transformaciones de nombre de pais a nombre estandarizado

**Iteraciones**
1. busqueda de nombres en la libreria pycountry
    1. busqueda del tipo fuzzy incluida en la libreria
    2. busqueda a partir si el nombre del pais esta contenido en el string
2. reglas duras
3. comparacion con thefuzz the nombres de paises
4. marcacion manual

In [ ]:
# Crea la nueva columna de valores estandarizados
data["standard_job_country"] = data["job_country"].str.replace(".","", regex=True).str.replace(",","", regex=True)
data["standard_job_country"] = data["job_country"].str.strip().str.upper()

In [ ]:
country_test = data.groupby("job_country")["Timestamp"].count().sort_index().index


search_name = []
for x in country_test:    
    
    try:
        name = pycountry.countries.search_fuzzy(x)[0].name
        search_name.append(name.upper())
    except:
        
        search_name.append("non_valid")

In [ ]:
# Crea un dataframe con las transformacions
dict_test = dict(zip(country_test,search_name))
country_standar_df = pd.DataFrame([country_test.T, search_name]).T
country_standar_df.columns = ["original","standard"]

In [ ]:

found_country = []

for x in country_standar_df[country_standar_df["standard"] == "non_valid"]["original"]:
    
    arr_countries = [x.name.upper() for x in pycountry.countries]
    for i, c in enumerate(arr_countries):

        if c in x:            
            found_country.append(c)
            
            break
        else:

            continue
    if i+1 == len(arr_countries):
        found_country.append("non_valid")
    else:
        continue

In [ ]:
country_standar_df.loc[country_standar_df["standard"] == "non_valid", "standard"] = found_country

In [ ]:
c0 = country_standar_df["standard"] == "non_valid"
c1 = country_standar_df.loc[country_standar_df["standard"] == "non_valid", "original"].str.contains("UNITED ST")
country_standar_df.loc[c0&c1, "standard"] = "UNITED STATES"

c0 = country_standar_df["standard"] == "non_valid"
c1 = country_standar_df.loc[country_standar_df["standard"] == "non_valid", "original"].str.contains("USA")
country_standar_df.loc[c0&c1, "standard"] = "UNITED STATES"

c0 = country_standar_df["standard"] == "non_valid"
c1 = country_standar_df.loc[country_standar_df["standard"] == "non_valid", "original"].str.contains("ENGLAN")
country_standar_df.loc[c0&c1, "standard"] = "ENGLAND"


In [ ]:
c0 = country_standar_df["standard"] == "non_valid"
c1 = country_standar_df.loc[c0,"original"].apply(lambda x : fuzz.ratio(x,"UNITED_STATES")) > 60
country_standar_df.loc[c0&c1, "standard"] = "UNITED STATES"

In [ ]:
c0 = country_standar_df["standard"] == "non_valid"
c1 = country_standar_df.loc[c0,"original"].apply(lambda x : fuzz.ratio(x,"CANADA")) > 60
country_standar_df.loc[c0&c1, "standard"] = "CANADA"

In [ ]:
country_standar_df.loc[country_standar_df["standard"] == "non_valid", "original"].values

In [ ]:
dict_country_non_valid = {
       'BRASIL':"BRASIL",
       'I WORK FOR A UAE-BASED ORGANIZATION THOUGH I AM PERSONALLY IN THE US':"UNITED SATES",
       'NEDERLAND':"NETHERANDS",
       'SCOTLAND':"UNITED KINGDOM",
       'SCOTLAND UK':"UNITED KINGDOM", 
       'THE US':"UNITED STATES", 
       'UAE':"UAE",
       'UK BUT FOR GLOBALLY FULLY REMOTE COMPANY':"UNITED KINGDOM", 
       'UK FOR US COMPANY':"UNITED KINGDOM",
       'UK REMOTE':"UNITED KINGDOM", 
       'UNITED KINDOM':"UNITED KINGDOM", 
       'UNITES KINGDOM':"UNITED KINGDOM",
       'WALES (UK)':"UNITED KINGDOM", 
       'WALES UK':"UNITED KINGDOM"}


In [ ]:
c0 = country_standar_df["standard"] == "non_valid"
country_standar_df.loc[c0, "standard"] = country_standar_df.loc[c0, "original"].map(dict_country_non_valid, na_action="'ignore'")

In [ ]:
country_standar_df["standard"] = country_standar_df["standard"].fillna("non_valid")
country_standar_df.loc[country_standar_df["standard"] == "non_valid", "original"].values

#### Transformacion de los datos de pais de original a estandar

In [ ]:
data["standard_job_country"] = data["job_country"].map(dict(zip(country_standar_df["original"],country_standar_df["standard"])))

In [ ]:
data.groupby(["standard_job_country"])["Timestamp"].count().sort_values(ascending=False)

## Estandarizar US states

estandarizacion de los nombres de los estados de estados unidos, se convierten a mayusculas y se toma el primer valor que se encuentre antes de una coma

In [ ]:
c0 = data["standard_job_country"] == "UNITED STATES"

data.loc[:,"standard_us_state"] = data.loc[:,"us_state"].str.upper()
data.loc[:,"us_state"] = data.loc[c0,"us_state"].sort_values().str.split(",").str[0]

len(data.loc[c0,"us_state"].unique())

## Estandarizacion ciudades

prmiero se busca la informacion correspondiente a las ciudades americanas, luego de estandarizar estas ciudades se procede a estandarizar las ciudades del resto del mundo

In [ ]:
data["job_city"] = data["job_city"].str.strip()

print("cantidad de entradas sin manipular")
len(data["job_city"].unique())

In [ ]:
c0 = data["standard_job_country"] == "UNITED STATES"

print("cantidad de ciudades totales")
print(len(data.loc[:,"job_city"].str.upper().unique()))
print("cantidad de ciudades de estados unidos")
print(len(data.loc[c0,"job_city"].str.upper().unique()))

### Busqueda de ciudades americanas

In [ ]:
# Lectura de informacion de ciudades americanas

us_cities_dir = "data\\us_cities.csv"
us_cities_names_df = pd.read_csv(us_cities_dir, sep="|")

us_cities_names_df["City"] = us_cities_names_df["City"].str.upper()
us_cities_names_df["State full"] = us_cities_names_df["State full"].str.upper()
us_cities_names_df = us_cities_names_df[["City","State short","State full"]].drop_duplicates()
us_cities_names_df

In [ ]:
c0 = data["standard_job_country"] == "UNITED STATES"


data["standard_job_city"] = "non_valid"

for state in data.loc[c0,"us_state"].unique():
    #print(state)

    #datos
    c1 = data.loc[c0,"us_state"] == state
    #referencias
    f1 = us_cities_names_df["State full"] == state

    # en estados unidos el estado x 
    c = c0 & c1
    
    
    for ciudad in us_cities_names_df.loc[f1,"City"]:
        #print(ciudad+" "+state)
        c2 = data.loc[c,"standard_job_city"] == "non_valid"

        test_fuzz_1 = data.loc[c&c2,"job_city"].apply(lambda x : fuzz.ratio(str(x).upper(),ciudad.upper())) > 60
        test_fuzz_2 = data.loc[c&c2,"job_city"].apply(lambda x : fuzz.token_set_ratio(str(x).upper(),ciudad.upper())) > 60

        test_fuzz = test_fuzz_1 | test_fuzz_2

        data.loc[c&c2&test_fuzz,"standard_job_city"] = ciudad


In [ ]:
c0 = data["standard_job_country"] == "UNITED STATES"


iter_cities = us_cities_names_df["City"].drop_duplicates().sort_values()

for ciudad in iter_cities: #us_cities_names_df.loc[:,"City"]:
    c1 = data.loc[c0,"standard_job_city"] == "non_valid"
    #print(ciudad)
    c = c0 & c1
    test_fuzz_1 = data.loc[c,"job_city"].apply(lambda x : fuzz.ratio(str(x).upper(),ciudad)) > 60
    test_fuzz_2 = data.loc[c,"job_city"].apply(lambda x : fuzz.token_set_ratio(str(x).upper(),ciudad)) > 60

    test_fuzz = test_fuzz_1 | test_fuzz_2
    if test_fuzz.sum() > 0:
        data.loc[c&test_fuzz,"standard_job_city"] = ciudad

### Estandarizacion de nombres de ciudades del resto del mundo

se tienen e cuenta solo las ciudades que segun el dataset tienen una poblacion mayor a 3.961325e4 (numero a priori) despues de obtener las descriptivas del set de datos

In [ ]:
dir_world_cities = "data\worldcities.csv"
#dir_world_cities = "data\country_capitals.csv"

world_cities_df = pd.read_csv(dir_world_cities, sep=",")

world_cities_df.sort_values(by = "population", ascending= False).describe()

wc0 = world_cities_df["population"] >= 3.961325e4	
world_cities_df = world_cities_df[wc0]

world_cities_df

In [ ]:
c0 = data["standard_job_country"] != "UNITED STATES"


#iter_cities = world_cities_df["city"].drop_duplicates().sort_values().str.upper()
iter_cities = world_cities_df["city"].drop_duplicates().sort_values().str.upper()

for ciudad in iter_cities: #us_cities_names_df.loc[:,"City"]:
    c1 = data.loc[c0,"standard_job_city"] == "non_valid"
    #print(ciudad)
    c = c0 & c1
    test_fuzz_1 = data.loc[c,"job_city"].apply(lambda x : fuzz.ratio(str(x).upper(),ciudad)) > 60
    test_fuzz_2 = data.loc[c,"job_city"].apply(lambda x : fuzz.token_set_ratio(str(x).upper(),ciudad)) > 60

    test_fuzz = test_fuzz_1 | test_fuzz_2

    if test_fuzz.sum() > 0:
        data.loc[c&test_fuzz,"standard_job_city"] = ciudad

## Estandariza annual salary

convertir a numerica la informacion

In [ ]:
data["standard_annual_salary"] = data["annual_salary"].str.replace(",","",regex=False).\
                                    str.replace("$","",regex=False).\
                                    astype(int)


data["total_income"] = data["monetary_compensation"] + data["standard_annual_salary"]

## Union de tasas de cambio y caculo de salarios en pesos

se realiza la union del valor de **currency** con su correspondiente conversion a **COP** luego de esto se procede a calcular las variables


| Nombre Campo | 
| --- | 
|total_income_cop|
|standard_annual_salary_cop|
|monetary_compensation_cop|

In [ ]:
dir_divisas = "data\currency.csv"

currency_df = pd.read_csv(dir_divisas)
currency_df["value_usd"] = currency_df["value_usd"].astype(float)
currency_df["value_cop"] = currency_df["value_cop"].astype(float)

In [ ]:
dict_currency = dict(zip(currency_df["name"],currency_df["value_cop"]))

data["currency_to_cop"] = data["standard_currency"].map(dict_currency, na_action=0)
data["currency_to_cop"] = data["currency_to_cop"].fillna(0)

In [ ]:
data["total_income_cop"] = data["currency_to_cop"] * data["total_income"]
data["standard_annual_salary_cop"] = data["currency_to_cop"] * data["standard_annual_salary"]
data["monetary_compensation_cop"] = data["currency_to_cop"] * data["monetary_compensation"]

# Export Final

In [ ]:
data.to_csv("salary_survey_cleaned.csv", index = False)

In [ ]:
for (k,v), dt in zip(dict_columns.items(),data.dtypes):
    print("|" + str(k) + "|" + str(v) + "|")
    #print("|" + str(k) + "|" + str(dt) + "|")

In [ ]:
for c in data.columns:
    print("|" + str(c) + "|  " + "  |")

# Documentacion modelado de datos

## Descripcion de variables

1. Variables en base de datos original.

| Nombre Original | tipo_dato | Descripción |
| --- | --- | --- |
|Timestamp|texto| Hora de la encuesta |
|How old are you?|texto| rango de edad del encuestado |
|What industry do you work in?|texto| categoria de industria donde trabaja el encuestado |
|Job title|texto| Nombre del trabajo |
|If your job title needs additional context, please clarify here:|texto| aclaraciones sobre el trabajo que desempeña en encuestado |
|What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)| texto | salario anual del encuestado |
|How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.|float64| ingresos adicionales como bonificaciones adicionales al salario |
|Please indicate the currency|texto| moneda en la que está el valor del salario anual |
|If "Other," please indicate the currency here: |texto| especificación para otras monedas en la que se suministra el salario |
|If your income needs additional context, please provide it here:|texto| aclaraciones sobre el ingreso reportado |
|What country do you work in?|texto| país en donde trabaja el encuestado |
|If you're in the U.S., what state do you work in?|texto| Si el país donde trabaja es EEUU especifique en qué estado (Ej. Florida)|
|What city do you work in?|texto| ciudad en donde trabaja el encuestado |
|How many years of professional work experience do you have overall?|texto| años de experiencia profesional, puede ser en otras industrias distintas a la indicada en la encuesta|
|How many years of professional work experience do you have in your field?|texto| años de experiencia profesional en la industria mencionada en la encuesta|
|What is your highest level of education completed?|texto| nivel de educación del encuestado|
|What is your gender?|texto| género del encuestado |
|What is your race? (Choose all that apply.)|texto| raza del encuestado|



2. Nombres de campos Homologados

| Nombre Original | Nombre Homologado |
| --- | --- |
|Timestamp|Timestamp|
|How old are you?|Age|
|What industry do you work in?|Industry|
|Job title|Job_title|
|If your job title needs additional context, please clarify here:|Job_additional_context|
|What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)|annual_salary|
|How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.|monetary_compensation|
|Please indicate the currency|currency|
|If "Other," please indicate the currency here: |other_currency|
|If your income needs additional context, please provide it here:|income_aditional_context|
|What country do you work in?|job_country|
|If you're in the U.S., what state do you work in?|us_state|
|What city do you work in?|job_city|
|How many years of professional work experience do you have overall?|professional_experience|
|How many years of professional work experience do you have in your field?|field_experience|
|What is your highest level of education completed?|highest_education_level|
|What is your gender?|gender|
|What is your race? (Choose all that apply.)|race|

3. Nuevos campos calculados

| Nombre Campo | Descripcion | tipo |
| --- | --- | --- |
|standard_currency| Estandarización de los valoers originales de la moneda   | Texto |
|standard_job_country| Estandarización del nombre del país donde trabaja el encuestado   | Texto |
|standard_us_state|  Estandarización del estado de EEUU donde trabaja el encuestado  | Texto |
|standard_job_city|  Estandarización de la ciudad donde trabaja el encuestado  | Texto |
|standard_annual_salary|  Estandarizacion del valor del ingreso reportado  | Numerico |
|total_income|  Suma del standard_annual_salary + monetary_compensation para obtener el total de ingresos por encuestado  | Numerico |
|currency_to_cop|  Valor del una unidad de la moneda reportada a cop Ej. 1 USD -> COP 4900  | Numerico |
|total_income_cop|  Conversión del total_income a COP total_income * currency_to_cop  |  Numerico |
|standard_annual_salary_cop|  Conversión a COP de la variable standard_annual_salary. standard_annual_salary * currency_to_cop  |  Numerico |
|monetary_compensation_cop|  Conversión a COP de la variable monetary_compensation. monetary_compensation* currency_to_cop |  Numerico |


## Proceso de actualizacion del reporte

1. Actualizacion de divisas

para actualizar las divisas se puede utilizar google sheets para generar una tabla de informacion actualizada, para ello generar un documento el cual debe tener las columnas 

* Name: codigo de 3 letras de las distintas divisas (ISO 4217)
* value_usd: formula de google sheets : =ifna(GOOGLEFINANCE(CONCATENATE("CURRENCY:",B3,"USD"))*A3,0)
* value_cop: =ifna(GOOGLEFINANCE(CONCATENATE("CURRENCY:USDCOP"))*C3,0)

**Ej.**

|Name|value_usd|value_cop|
| --- | --- | --- |
|GBP | 1.2055 | 5717.691322 |


una vez se tenga este documento se procede a descargarlo y guardarlo en la carpeta **data** con el nombre **currency.csv**

2. Estandarizacion de informacion 
Descargar el resultado de la encuesta y almacenarlo en la carpeta **data** del proyecto y verificar que tenga el nombre 
**Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv**

Ejecutar el notebook en su totalidad, el cual consta de las siguientes secciones
- Estandarizar Nombres de campos
- Estandarización currency
- Estandarización Paises
- Estandarizar US states
- Estandarizacion ciudades
- Estandariza annual salary
- Union de tasas de cambio y caculo de salarios en pesos

3. Descargar la informacion de la encuesta estandarizada

    El procesamiento de los datos queda almacenadoen el archivo **salary_survey_cleaned.csv**

4. cargar la informacion en looker studio 


